# Project Phân tích và ứng dụng máy học sự kiện chìm tàu Titanic
<p align="center">
  <img src="Titanic.jpg" alt="Titanic" width="400"/>
</p>
bối cảnh: Titanic là chiếc tàu vượt đại dương chở khách chạy bằng động cơ hơi nước đã đi vào lịch sử ngành hàng hải vì vụ tai nạn hàng hải đã xảy ra với nó cũng như những bí ẩn liên quan. Tên chính thức của nó là RMS Titanic (RMS là viết tắt của Royal Mail Ship). Tàu bắt đầu được đóng vào năm 1909 và được hạ thủy năm 1912. Là con tàu lớn, hiện đại, lộng lẫy và sang trọng nhất lúc đó, Titanic mang theo tham vọng thống trị tuyến đường biển xuyên Đại Tây Dương của công ty sở hữu nó, hãng vận tải biển The White Star Line. Tuy nhiên, trong chuyến vượt Đại Tây Dương đầu tiên và cũng là cuối cùng của nó vào tháng 4 năm 1912, Titanic đã đắm do đâm vào một tảng băng trôi, khiến hơn 1.500 người tử nạn. Vụ đắm tàu này đã đi vào lịch sử như là vụ tai nạn hàng hải nghiêm trọng nhất trong thời bình. (Wikipedia)

## Các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd